In [1]:
import os
import pandas as pd
import numpy as np
import shutil
import cPickle as pkl

DATASET_VERSION = 2
ROOT_DIR = '../../vehicles_dataset_v{}'.format(DATASET_VERSION)
IMAGES_DIR = os.path.join(ROOT_DIR, 'images')
ANNOTATIONS_DIR = os.path.join(ROOT_DIR, 'annotations')

## initialize new dir tree

In [2]:
try:
    os.makedirs(IMAGES_DIR)
    os.makedirs(ANNOTATIONS_DIR)
except:
    pass

### crawl dirs and rename csv files, images to include original image name
### move all pics to 'images' dir and all csvs to 'annotations' dir

In [3]:
move_list=[]
empty_folders=[]
for root,_,files in os.walk(ROOT_DIR):
    if root==IMAGES_DIR or root==ANNOTATIONS_DIR or len(files)==0:
        continue
    source_image_name = root.split(os.sep)[-1].replace('crop','')
    for file in files:
        src = os.path.join(root,file)
        if file == 'Images.csv':
            dst = os.path.join(ANNOTATIONS_DIR,source_image_name+'.csv')
        else:
            patch_id = int(os.path.splitext(file)[0].replace('pic',''))
            new_filename = '{}_{:05}.jpg'.format(source_image_name, patch_id)
            dst = os.path.join(IMAGES_DIR, new_filename)
        move_list.append((src,dst))
    empty_folders.append(root)

# move pics and csvs
for src, dst in move_list:
    shutil.move(src,dst)
print 'Moved {} files.'.format(len(move_list))
# del empty folders
for fldr in empty_folders:
    shutil.rmtree(fldr)

Moved 1488 files.


## each large sweep was cut into smaller pictures, these pictures are called 'patches' from here on out.

In [4]:
classes = ('__background__',
           'Car', 'Van', 'Truck',
           'ConcreteTruck', 'Bus')
num_classes = len(classes)
class_to_ind = dict(zip(classes, xrange(num_classes)))

def correct_alignment(raw_points):
    raw_points[raw_points < 1] = 1
    raw_points[raw_points > 900] = 900
    return

def get_shapes(patch_df):
    raw_points = np.array(patch_df.ix[:,1:], dtype=np.float32)[:,::-1]
    correct_alignment(raw_points)
    polygons = np.zeros((raw_points.shape[0],4,2), dtype=np.float32)
    bboxes = np.zeros((raw_points.shape[0],4), dtype=np.float32)
    for i in xrange(raw_points.shape[0]):
        poly = raw_points[i].reshape(4,2) - 1 # zero-index points
        polygons[i]=poly
        bboxes[i,0:2]=np.min(poly, 0) # get xmin, ymin
        bboxes[i,2:4]=np.max(poly, 0) # get xmax, ymax
    return bboxes, polygons

def get_classes(patch_df):
    return patch_df['Entities EntityType'].values.tolist()

## the following script parses this 'patch db' from all original csv files using pandas

In [5]:
from glob import glob
csvs = glob(os.path.join(ANNOTATIONS_DIR, '*.csv'))
patch_db={}
s=set()

for csv in csvs:
    source_image_name = os.path.basename(os.path.splitext(csv)[0])
    df = pd.read_csv(csv)
    
    # weed out all invalid boxes per patch
    patch_list = df['FileName']
    i = 0
    while i<len(patch_list) and isinstance(patch_list[i],str):
        j=1
        while i+j<len(patch_list) and not isinstance(patch_list[i+j],str):
            j+=1
        
        # set flattened patch name
        patch_id = int(os.path.splitext(patch_list[i])[0].replace('pic',''))
        patch_name = '{}_{:05}.jpg'.format(source_image_name, patch_id)
        patch_df = df.ix[i:i+j-1, 'Entities EntityType':'Entities P1 X'].reset_index(drop=True)
        first_entry = patch_df.ix[0,'Entities EntityType']
        
        # log bboxes and polygons in patch to patch db. skip non-existing pics and patches with no labels.
        if isinstance(first_entry,str) and os.path.exists(os.path.join(IMAGES_DIR,patch_name)):
            bboxes, polygons = get_shapes(patch_df)
            gt_classes = get_classes(patch_df)
            patch_db[patch_name]= {'boxes' : bboxes,
                                   'polygons' : polygons,
                                   'gt_classes' : gt_classes}
        # advance loop
        i+=j

In [6]:
# check for nans
for _,d in patch_db.iteritems():
    for k,v in d.iteritems():
        if k == 'boxes' or k == 'polygons':
            if v is not None and (np.isnan(v).any() == True or np.isnan(v).any() == True):
                print('found nan in: ')
                print(k,v)
print 'reached end'

reached end


In [7]:
# non-empty patches
nep = [k for k in patch_db.keys() if patch_db[k]['boxes'] is not None]
print 'non-empty patches: {}'.format(len(nep))
# patches total
print 'patches total: {}'.format(len(patch_db))

non-empty patches: 1225
patches total: 1225


In [8]:
# save to disk
with open(os.path.join(ANNOTATIONS_DIR, 'complete_dataset_v{}.pkl'.format(DATASET_VERSION)), 'wb') as f:
    pkl.dump(patch_db, f)

## visualize bboxes in patches

In [9]:
import cv2
import os
import pandas as pd
import numpy as np
import shutil
import cPickle as pkl

DATASET_VERSION = 2
ROOT_DIR = '../../vehicles_dataset_v{}'.format(DATASET_VERSION)
IMAGES_DIR = os.path.join(ROOT_DIR, 'images')
ANNOTATIONS_DIR = os.path.join(ROOT_DIR, 'annotations')

def vis_detections(im, title, roidb): #tp, fp, fn):
    """Visual debugging of detections."""
    import matplotlib.pyplot as plt
    im = im[:, :, (2, 1, 0)]
    plt.cla()
    plt.title(title)
    plt.imshow(im)
    _draw_detections(im, roidb, plt)
    plt.show()

def _draw_detections(im, roidb, plt):
    boxes = roidb['boxes']
    if boxes is None: return
    for i in xrange(boxes.shape[0]):
        bbox = boxes[i, :4]
        plt.gca().add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2] - bbox[0],
                          bbox[3] - bbox[1], fill=False,
                          edgecolor='red', linewidth=2)
            )
    return

In [10]:
# load patch_db
with open(os.path.join(ANNOTATIONS_DIR, 'complete_dataset_v{}.pkl'.format(DATASET_VERSION))) as f:
    patch_db = pkl.load(f)

# visualize annotations
for k,v in patch_db.iteritems():
    image_path = os.path.join(IMAGES_DIR, k)
    im = cv2.imread(image_path)
    if im is None:
        print("couldn't load image: {}".format(image_path))
        continue
#     print(image_path)
#     vis_detections(im, k, v)

## split patch_db into train,val,test sets

In [11]:
import sklearn.cross_validation as cv

ss = cv.ShuffleSplit(len(patch_db), n_iter=1, test_size=0.15)
for t1, t2 in ss:
    ss2 = cv.ShuffleSplit(len(t1), n_iter=1, test_size=0.15)
    for r1, r2 in ss2:
        train = t1[r1]
        val = t1[r2]
    test = t2

with open(os.path.join(ANNOTATIONS_DIR, 'splits_indices_v{}.pkl'.format(DATASET_VERSION)), 'wb') as f:
    pkl.dump({'train': train,
              'val': val,
              'test': test}, f)

In [12]:
def get_split_from_ds(ds, idx):
    split = {}
    keys = ds.keys()
    for j in xrange(len(idx)):
        k = keys[idx[j]]
        split[k] = ds[k]
    return split

In [13]:
dataset_file = os.path.join(ANNOTATIONS_DIR, 'complete_dataset_v{}.pkl'.format(DATASET_VERSION))
splits_file = os.path.join(ANNOTATIONS_DIR, 'splits_indices_v{}.pkl'.format(DATASET_VERSION))

if os.path.exists(dataset_file):
    with open(dataset_file) as f:
        patch_db = pkl.load(f)

    if os.path.exists(splits_file):
        # load splits indices
        with open(splits_file) as f:
            d = pkl.load(f)
            
        train_ds = get_split_from_ds(patch_db, d['train'])
        val_ds = get_split_from_ds(patch_db, d['val'])
        test_ds = get_split_from_ds(patch_db, d['test'])

## get stats from splits

In [14]:
def count_boxes(ds):
    count = 0
    for k,v in ds.iteritems():
        count += len(v['gt_classes'])
    return len(ds), count

In [15]:
print('stats for vehicle dataset, version: {}\n-------'.format(DATASET_VERSION))
print('train patches: {}, vehicles: {}'.format(*count_boxes(train_ds)))
print('val patches: {}, vehicles: {}'.format(*count_boxes(val_ds)))
print('test patches: {}, vehicles: {}'.format(*count_boxes(test_ds)))

stats for vehicle dataset, version: 2
-------
train patches: 884, vehicles: 3625
val patches: 157, vehicles: 654
test patches: 184, vehicles: 759
